In [1]:
# My personal AI which works on voice command and face recognisation to perform basic tasks and behave as an AI chat bot

In [2]:
# !pip install 

In [3]:
import win32com.client
import speech_recognition as sr
import pyaudio
import wikipedia
import webbrowser
import os
import openai
from nltk.sentiment import SentimentIntensityAnalyzer
import time
import numpy 
import requests
import json
import cv2
import face_recognition
import smtplib
import matplotlib.pyplot as plt
from datetime import datetime
from deepface import DeepFace
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import numpy as np

query = None   # Global Variables

In [4]:
# todo: to send gmail to anyone

def send_mail(to,content):    

    server = smtplib.SMTP("smtp.gmail.com","your_code")

    server.ehlo()
    server.starttls()
    server.login("your_email@gmail.com","cutp fhrd lclp dwyc")
    server.sendmail("your_email@gmail.com",to,content)
    server.close()

In [5]:
# todo: set the voice for max

speaker = win32com.client.Dispatch("SAPI.SpVoice")

voice_name = speaker.GetVoices()

print(voice_name.Item(0).GetAttribute ("Name"))

speaker.Voice

speaker.SetVoice(voice_name.Item(0))

Microsoft David Desktop


In [6]:
# todo: returns text string from voice command

def takecommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\nListening.....")
        r.pause_threshold = 1
        audio = r.listen(source)
        
    try:
        print("Recognising.....")
        query = r.recognize_google(audio,language='en-in')
        print(f"\nHrithik: {query}\n")

    except Exception as e:
        print("\nSay that again please....")
    return query

In [7]:
# todo: to get date and time

def time_date():

    timestamp = time.asctime()

    timestamp_list = timestamp.split(" ")

    return timestamp_list

In [8]:
# todo: to get real-time weather report

def get_weather(location):

    param = {
            'q': location,
            'appid': "your_api_key",
            'units': 'imperial'              # to get the temp in F
             }

    weather = requests.get("http://api.openweathermap.org/data/2.5/weather",params=param)

    data = weather.json()

    if weather.status_code == 200:
        print(data)
        temperature = data['main']['temp']
        description = data['weather'][0]['description']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        print(f'Temperature: {round(((int(temperature)-32)*5)/9, 1)} C')     # converted the temp to C
        print(f'Description: {description}')
        print(f'Humidity: {humidity}%')
        print(f'Wind Speed: {wind_speed} m/s')
        
        

In [9]:
def face_detect():
    # Load the pre-trained Haar cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Load a sample image of yourself
    your_image = face_recognition.load_image_file("C:/Users/asus/OneDrive/Desktop/WhatsApp Image 2024-03-06 at 12.42.55_07df780f.jpg")
    your_encoding = face_recognition.face_encodings(your_image)[0]

    # Initialize MediaPipe hands module
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
    mp_drawing = mp.solutions.drawing_utils

    # Initialize pycaw for volume control
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    # Define a function to classify hand gestures
    def classify_hand_gesture(landmarks):
        thumb_is_open = landmarks[4].y < landmarks[3].y
        index_is_open = landmarks[8].y < landmarks[6].y
        middle_is_open = landmarks[12].y < landmarks[10].y
        ring_is_open = landmarks[16].y < landmarks[14].y
        pinky_is_open = landmarks[20].y < landmarks[18].y

        current_volume = volume.GetMasterVolumeLevelScalar()

        if thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
            new_volume = min(current_volume + 0.0125, 1.0)  # Increase volume
            volume.SetMasterVolumeLevelScalar(new_volume, None)
            return "Thumbs Up"
        elif not thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
            new_volume = max(current_volume - 0.0125, 0.0)  # Decrease volume
            volume.SetMasterVolumeLevelScalar(new_volume, None)
            return "Thumb Down"
        return None

    # Initialize the camera
    video = cv2.VideoCapture(0)

    # Custom thresholds for each emotion
    custom_thresholds = {
        'angry': 0.2,
        'disgust': 0.05,
        'fear': 0.4,
        'happy': 0.6,
        'sad': 0.25,
        'surprise': 0.35,
        'neutral': 0.4
    }

    while True:
        __, frame = video.read()
        
        # Find all face locations in the current frame
        face_locations = face_recognition.face_locations(frame)

        # If faces are found, try to recognize them
        if face_locations:
            face_encodings = face_recognition.face_encodings(frame, face_locations)
            for face_encoding in face_encodings:
                # Compare the current face with the known face
                match = face_recognition.compare_faces([your_encoding], face_encoding)
                
                print('Match Result',match[0])
                
                if match[0]:
                    detect = match
                    
                else:
                    detect = match

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        # Process the image and find hands
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Draw hand landmarks and classify gestures
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = classify_hand_gesture(hand_landmarks.landmark)
                if gesture:
                    cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract the face region from the grayscale frame
            face_roi = gray[y:y+h, x:x+w]

            try:
                # Convert the face region to RGB format (DeepFace requires RGB input)
                face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2RGB)

                # Perform emotion analysis on the face region
                analyze = DeepFace.analyze(face_rgb, actions=['emotion'], enforce_detection=False)
                
                #Print the analyze result
                print('Analyze Result:',analyze)
                
                #Check if the result is as expected
                if 'emotion' in analyze:
                    emotions = analyze['emotion']
                    print("Emotions detected:", emotions)
                    
                    #Filter emotions based on custom threshold
                    dominant_emotions = [emotion for emotion, score in analyze['emotion'].items() if score > custom_thresholds.get(emotion, 0.3)]
                    print('Dominant emotions:',dominant_emotions) 
                    
                    # Display the filtered dominant emotions on the frame
                    if dominant_emotions:
                        cv2.putText(frame, ', '.join(dominant_emotions), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw a rectangle around the detected face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            except Exception as e:
                print('Error analyzing face:', e)

        # Display the resulting frame with detected faces and emotions
        cv2.imshow('Face Emotion and Gesture Detection', frame)

        # Check for exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    video.release()
    cv2.destroyAllWindows()


In [10]:
if __name__ == "__main__":
    
    face_detect()
    print(detect)
    
    if detect:
        
        print('Face Recognised')
        
        # dictionary to store esentinal apps name and location on PC
        app_dic = {'code':"C:/Program Files/Microsoft Visual Studio/2022/Community/Common7/IDE/devenv.exe",
                'chrome':"C:/Program Files/Google/Chrome/Application/chrome.exe",
                'excel':"C:/Program Files/Microsoft Office/root/Office16/EXCEL.EXE",
                'word':"C:/Program Files/Microsoft Office/root/Office16/WINWORD.EXE",
                'powerpoint':"C:/Program Files/Microsoft Office/root/Office16/POWERPNT.EXE",
                'my pc':"C:/Users/asus/Desktop/My PC - Shortcut.lnk",
                'edge':"C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe"}   
        
        get_time = time_date() 
                                                  # to get through date and time function
        time = get_time[3].split(":")

        if (int(time[0])>=5 and int(time[0])<12):
            speaker.Speak("\nGood morning! Hrithik")

        elif (int(time[0])>=12 and int(time[0])<18):
            speaker.Speak("\nGood afternoon! Hrithik")

        elif (int(time[0])>18):
            speaker.Speak("\nGood evening! Hrithik")
        
        while 'good bye' not in query :

            #query = input("Enter your command: ")+sentiment_analyse_nltk(query)
            query = takecommand().lower()
            print(query)

            if 'send mail' in query:
                speaker.Speak("Please tell me to whom and what message want to send?")
                to = input("To: ")
                content = input("Message: ")
                
                try:
                    send_mail(to,content)
                    speaker.Speak("Mail has been send!")

                except exception as e:
                    print(e)
                    speaker.Speak("Sorry i am unable to send your mail right now")
                
            elif "open youtube" in query:
                speaker.Speak("Opening YouTube")
                webbrowser.open("youtube.com")

            elif "google chrome" in query:
                speaker.Speak("Opening Google Chrome")
                os.startfile(f"{app_dic['chrome']}")   
            
            elif "date" in query:
                speaker.Speak(f"{get_time[1] + get_time[2] + get_time[4]}")
                
            elif "time" in query:
                speaker.Speak(f"{get_time[3]}")

            elif "hello max" in query:

                print("\nHello Hrithik\nMax here!")

                while "bye max" not in query :

        #           query  = input("Enter your command: ")+sentiment_analyse_nltk(query)
                    query = takecommand().lower()

                    openai.api_key = "your_api_key"

                    chat_completion = openai.chat.completions.create(
                        messages=[
                            {
                                "role": "system",
                                "content": query,
                            }
                        ],
                        temperature=1,
                        max_tokens=4000,
                        top_p=0.5,
                        frequency_penalty=1,
                        presence_penalty=1,
                        model="gpt-3.5-turbo",
                    )

                    print(f"Max: {chat_completion.choices[0].message.content}")
                    speaker.Speak(chat_completion.choices[0].message.content)

            elif "code" in query:
                speaker.Speak("Opening VS Code")
                os.startfile(f"{app_dic['code']}")

            elif "excel" in query:
                speaker.Speak("Opening MS Excel")
                os.startfile(f"{app_dic['excel']}")

            elif "ms word" in query:
                speaker.Speak("Opening MS Word")
                os.startfile(f"{app_dic['word']}")

            elif "powerpoint" in query:
                speaker.Speak("Opening MS Powerpoint")
                os.startfile(f"{app_dic['powerpoint']}")

            elif 'my pc' in query:
                speaker.Speak("Opening My PC")
                os.startfile(f"{app_dic['my pc']}")

            elif 'edge' in query:
                speaker.Speak("Opening Micosoft Edge")
                os.startfile(f"{app_dic['edge']}")

            elif 'weather' in query:
                speaker.Speak("For which location?")
                get_weather(takecommand())
                
    else:
        print("Sorry! Face not recognised")
        speaker.Speak("Sorry! face not recognised")

Match Result False
Analyze Result: [{'emotion': {'angry': 59.53845977783203, 'disgust': 0.2146853832527995, 'fear': 8.455609530210495, 'happy': 16.20001345872879, 'sad': 3.3518854528665543, 'surprise': 0.3488729940727353, 'neutral': 11.890476197004318}, 'dominant_emotion': 'angry', 'region': {'x': 0, 'y': 0, 'w': 188, 'h': 188, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Match Result True
Analyze Result: [{'emotion': {'angry': 0.19348094938322902, 'disgust': 0.000290145430881239, 'fear': 97.144216299057, 'happy': 0.011848607391584665, 'sad': 2.166397124528885, 'surprise': 0.00026112304567504907, 'neutral': 0.4835110157728195}, 'dominant_emotion': 'fear', 'region': {'x': 15, 'y': 16, 'w': 160, 'h': 160, 'left_eye': (128, 68), 'right_eye': (51, 64)}, 'face_confidence': 0.99}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.3492432180792093, 'disgust': 0.00020875486370641738, 'fear': 5.1886942237615585, 'happy': 0.028775856480933726, 'sad': 78

Match Result True
Analyze Result: [{'emotion': {'angry': 0.5582136567682028, 'disgust': 9.036901360559568e-05, 'fear': 1.7736656591296196, 'happy': 0.0038846144889248535, 'sad': 67.83144474029541, 'surprise': 3.238985186726495e-05, 'neutral': 29.832670092582703}, 'dominant_emotion': 'sad', 'region': {'x': 10, 'y': 15, 'w': 167, 'h': 167, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0.96}]
Match Result True
Analyze Result: [{'emotion': {'angry': 1.1054717004299164, 'disgust': 0.004706549952970818, 'fear': 9.381932765245438, 'happy': 0.5840713623911142, 'sad': 34.112730622291565, 'surprise': 0.008348358096554875, 'neutral': 54.80273962020874}, 'dominant_emotion': 'neutral', 'region': {'x': 0, 'y': 0, 'w': 196, 'h': 196, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 2.540097013115883, 'disgust': 0.025112845469266176, 'fear': 36.80993318557739, 'happy': 4.588688164949417, 'sad': 31.39408826828003, 'surprise

Match Result True
Analyze Result: [{'emotion': {'angry': 0.3416802966967225, 'disgust': 1.8339528506938763e-08, 'fear': 0.17954421928152442, 'happy': 0.001231842179549858, 'sad': 1.192748174071312, 'surprise': 0.0018491913579055108, 'neutral': 98.28295111656189}, 'dominant_emotion': 'neutral', 'region': {'x': 0, 'y': 0, 'w': 151, 'h': 151, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.14066981897292363, 'disgust': 3.178964883930608e-05, 'fear': 0.5804174340268891, 'happy': 1.0659097101878774, 'sad': 6.568367072149166, 'surprise': 1.793766049297577, 'neutral': 89.85083578238914}, 'dominant_emotion': 'neutral', 'region': {'x': 12, 'y': 13, 'w': 148, 'h': 148, 'left_eye': (114, 63), 'right_eye': (47, 60)}, 'face_confidence': 0.95}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.1346880802884698, 'disgust': 8.930812356311435e-06, 'fear': 0.0893470598384738, 'happy': 0.0011071889275626745, 'sad': 0.8335757

Match Result True
Analyze Result: [{'emotion': {'angry': 2.0089255645871162, 'disgust': 0.0013937973562860861, 'fear': 18.297338485717773, 'happy': 0.015948958753142506, 'sad': 62.06425428390503, 'surprise': 0.012008026533294469, 'neutral': 17.60013699531555}, 'dominant_emotion': 'sad', 'region': {'x': 17, 'y': 14, 'w': 169, 'h': 169, 'left_eye': (131, 71), 'right_eye': (56, 73)}, 'face_confidence': 0.96}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.6027088966220617, 'disgust': 0.0004155150236329064, 'fear': 51.2542724609375, 'happy': 0.05608747014775872, 'sad': 31.727510690689087, 'surprise': 0.0030095075999270193, 'neutral': 16.355998814105988}, 'dominant_emotion': 'fear', 'region': {'x': 0, 'y': 0, 'w': 192, 'h': 192, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.17000163508559643, 'disgust': 9.617260720412017e-05, 'fear': 8.68217171047071, 'happy': 0.007630906293163071, 'sad': 78.58555572205118

Match Result True
Analyze Result: [{'emotion': {'angry': 0.36899447698957, 'disgust': 0.0011970022052142002, 'fear': 26.100298631155653, 'happy': 0.03706381097979117, 'sad': 46.08564125470623, 'surprise': 0.001859831132813464, 'neutral': 27.404945802099014}, 'dominant_emotion': 'sad', 'region': {'x': 0, 'y': 0, 'w': 192, 'h': 192, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.03038779250346124, 'disgust': 7.66087970838214e-08, 'fear': 5.630673468112946, 'happy': 21.16793841123581, 'sad': 3.673802688717842, 'surprise': 0.0440769421402365, 'neutral': 69.4531261920929}, 'dominant_emotion': 'neutral', 'region': {'x': 11, 'y': 25, 'w': 57, 'h': 57, 'left_eye': None, 'right_eye': None}, 'face_confidence': 1.0}, {'emotion': {'angry': 6.306951493024826, 'disgust': 0.07754788384772837, 'fear': 5.376023054122925, 'happy': 0.011102157441200688, 'sad': 37.51751184463501, 'surprise': 0.0029066219212836586, 'neutral': 50.70796

Match Result True
Analyze Result: [{'emotion': {'angry': 0.27964941691607237, 'disgust': 2.6219743176625343e-05, 'fear': 4.220813512802124, 'happy': 0.06098195444792509, 'sad': 60.01685857772827, 'surprise': 0.005348042759578675, 'neutral': 35.41631996631622}, 'dominant_emotion': 'sad', 'region': {'x': 13, 'y': 11, 'w': 163, 'h': 163, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0.97}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.0030724044336238876, 'disgust': 3.0777294046657744e-07, 'fear': 0.050787755753844976, 'happy': 97.5215196609497, 'sad': 0.262255291454494, 'surprise': 0.07991429301910102, 'neutral': 2.0824525505304337}, 'dominant_emotion': 'happy', 'region': {'x': 0, 'y': 0, 'w': 187, 'h': 187, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 1.4870412647724152, 'disgust': 0.010288682096870616, 'fear': 21.162772178649902, 'happy': 0.20725762005895376, 'sad': 61.95281744003296, 'surp

Match Result True
Analyze Result: [{'emotion': {'angry': 0.7603982904283154, 'disgust': 0.007605262541653221, 'fear': 26.31464914258112, 'happy': 0.006631479545104673, 'sad': 62.11717869248136, 'surprise': 0.0008360528314216179, 'neutral': 10.792705213153983}, 'dominant_emotion': 'sad', 'region': {'x': 0, 'y': 0, 'w': 186, 'h': 186, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.7366209264320688, 'disgust': 0.0020520033417415935, 'fear': 26.811657953957198, 'happy': 0.0732267726552798, 'sad': 44.42187216412259, 'surprise': 0.0003873452218477327, 'neutral': 27.954186675202052}, 'dominant_emotion': 'sad', 'region': {'x': 0, 'y': 0, 'w': 179, 'h': 179, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0}]
Match Result True
Analyze Result: [{'emotion': {'angry': 0.28087758836119314, 'disgust': 0.0022144865486032534, 'fear': 96.71741763145714, 'happy': 0.01713565230056318, 'sad': 2.679041267449705, 'surprise': 0

NameError: name 'detect' is not defined